Now take your Keras skills and go build another neural network. Pick your data set, but it should be one of abstract types, possibly even nonnumeric, and use Keras to make five implementations of your network. Compare them both in computational complexity as well as in accuracy and given that tradeoff decide which one you like best.

Working on the https://www.kaggle.com/moltean/fruits/ image-recognition dataset.

## Conclusions
CNN seems by far the best for this particular dataset.  It takes quite a while to process, especially on my meagre laptop, but it's really hard to argue with a 98% validation success rate.  Interestingly, it was the first two iterations I tried ("Convolutional Neural Network, v1" and "Convolutional Neural Network, v2") that produced the best and most reliable results, with val_acc scores of 98.49% and 98.58%.

## Seq MLP Conclusions:
Decent, and very fast (once you've uploaded the images) but not in the 90%+ category yet. 

Extra Epochs seems to do little, as by #10 it seems to have reached a point of minimal or negative returns, where the random associations in each iteration cause it to "forget" how to label some fruits even as it learns how to label others.

The most critical factor seems to be the image size, as a scale of 100 pixels (the scale of the images in the download for this collection) produces abysmal results, and gets progressively better until it reaches somewhere between 10 and 20, after which further decreases drop performance (likely due to simply not having enough identifying data to go on in each image).

## CNN conclusions:
The most effective of the three methods, though also the most time-intensive.

Increasing the kernel size seemed to reduce performance.  Optimal image size was somewhere in the vicinity of 30x30.  An increase in the number of epochs either reduced accuracy or had no effect other than to allow randomness to drop the result by a few fractions of a percent.

## RNN conclusions:
Almost as time-intensive as CNN, but with less accuracy to show for it.

Though increasing the number of epochs did improve performance, accuracy gains leveled off around the 90th percentile, and then proceeded to fall.

Dropping the image scale to 20x20 improved performance, as expected, though I'm uncertain how much of the sudden spike in val_acc in epoch 10 of v3 was due to a lucky random fluke or actual improvement in performance.  Epoch 9 saw a *drop* in val_acc, to .88, which then bounced up to .94 for epoch 10.  On the other hand, the accuracy (acc, not val_acc) of the model had steadily increased throughout.  The combination of these two implies to me that the model *was* becoming more accurate, and the last epoch simply saw a marked reduction in overfitting.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import math
from IPython.display import display
from timeit import default_timer as timer
import os
import pydotplus
from itertools import cycle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

%matplotlib inline
sns.set_style('white')

In [2]:
import sklearn

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
from sklearn import neighbors
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import LabelEncoder, Imputer
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.cluster import MiniBatchKMeans
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation
from sklearn import metrics

from sklearn import tree
from sklearn import preprocessing


In [3]:
# Keras and Tensorflow
import tensorflow as tf
import keras

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop, SGD, adam
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Import the backend
from keras import backend as K

# import PIL
import PIL as pil
from PIL import Image


Using TensorFlow backend.


In [4]:
#KERAS
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
from PIL import Image
from numpy import *

# SKLEARN
from sklearn.utils import shuffle


In [5]:
# verify that we have the folder in the correct place, and what the subfolders are labeled
print(os.listdir("fruits-360"))

['Tinker_Test', '.DS_Store', 'LICENSE', 'Test', 'papers', 'Tinker_Train', 'Training', 'readme.md', 'test-multiple_fruits']


In [6]:
# function that, given a folder name, will return:
    # 1) the datapath for that folder and 
    # 2) a list of the folders inside

def get_datapath(folder):
    data_path = 'fruits-360/' + str(folder) + '/'
    fruits_list = os.listdir(data_path) 
    if '.DS_Store' in fruits_list:
        fruits_list.remove('.DS_Store')
    return(data_path, fruits_list)


# function that, given a folder and a number (X) of fruit-folders to pick, returns: 
    # 1) a list of the image pathways in the selected folders (folder_image_list)
    # 2) a list of the fruit labels for each of those images (image_label_list)

def get_image_list(folder,num_fruits):
    data_path, fruits_list = get_datapath(folder)
    
    folder_image_list = []
    image_label_list = []
    image_numlabel_list = []
    fruit_counter = 0
    
    fruit = fruits_list[0]
    data_path_fruit = str(data_path + str(fruit) + '/')
    image_list_fruit = os.listdir(data_path_fruit) 

    for fruit in fruits_list[:num_fruits]:
        data_path_fruit = str(data_path + str(fruit) + '/')
        image_list_fruit = os.listdir(data_path_fruit)
        
        for image in image_list_fruit:
            image_address = str(data_path_fruit + str(image))
            folder_image_list.append(image_address)
            image_label_list.append(fruit)
            image_numlabel_list.append(fruit_counter)

        fruit_counter += 1

    return(folder_image_list, image_label_list, image_numlabel_list)

# could also use this format datapath.split("/")[-1] to grab the fruit label of a image


# function that, given a folder and a number (X) of fruit-folders to pick, returns: 
    # 1) a list of the image pathways in the selected folders (folder_image_list)
    # 2) a list of the fruit labels for each of those images (folder_label_list)
    # 3) a list of arrays representing each image in the list of image pathways (image_array_list)

def get_image_arrays(folder, num_fruits, image_scale):
    print('Working on ' + folder + ' images...', end = '')

    folder_image_list, folder_label_list, folder_numlabel_list = get_image_list(folder,num_fruits)
    image_array_list = []
    
    for image in folder_image_list:
        image_jpg = load_img(image, target_size=(image_scale, image_scale))
        image_array = img_to_array(image_jpg)
        image_array /= 255
        image_array_list.append(image_array)

    print('  Done with upload and initial processing.')
    return(folder_image_list, folder_label_list, folder_numlabel_list, image_array_list)

## Sequential MLP

In [76]:
# Fix our settings

number_fruits = 100
image_scale = 15

sample_list = [.01,.02,.03,.04,.05, .1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

In [77]:
# Work through the test dataset

test_datapaths, test_labels, test_numlabels, test_images = get_image_arrays('Test', 
                                                                            number_fruits, 
                                                                            image_scale)
test_sample = []
for x in sample_list:
    test_sample.append(test_images[int(x*len(test_images))])  # takes a snapshot of a few images before flattening

for x in range(0,len(test_images)):
    test_images[x] = np.ndarray.flatten(test_images[x])

test_images = np.array(test_images)

y_test = keras.utils.to_categorical(test_numlabels, number_fruits)


Working on Test images...  Done with upload and initial processing.


In [78]:
# Work through the train dataset

train_datapaths, train_labels, train_numlabels, train_images = get_image_arrays('Training', 
                                                                                number_fruits, 
                                                                                image_scale)
train_sample = []
for x in sample_list:
    train_sample.append(train_images[int(x*len(train_images))])

for x in range(0,len(train_images)):
    train_images[x] = np.ndarray.flatten(train_images[x])

train_images = np.array(train_images)

y_train = keras.utils.to_categorical(train_numlabels, number_fruits)


Working on Training images...  Done with upload and initial processing.


In [79]:
# boil down the size of the input arrays for the model
input_shape = str(train_images[1].shape)
input_shape = input_shape.replace('(','')
input_shape = input_shape.replace(',)','')
input_shape = int(input_shape)
print(input_shape)

675


In [80]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(input_shape,)))  
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(number_fruits, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 64)                43264     
_________________________________________________________________
dropout_25 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_26 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 100)               6500      
Total params: 53,924
Trainable params: 53,924
Non-trainable params: 0
_________________________________________________________________


In [81]:
history = model.fit(train_images, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(test_images, y_test))
score = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/10
51770/51770 [==============================] - 2s 47us/step - loss: 2.6653 - acc: 0.3020 - val_loss: 1.1949 - val_acc: 0.7096
Epoch 2/10
51770/51770 [==============================] - 2s 34us/step - loss: 1.1461 - acc: 0.6371 - val_loss: 0.7102 - val_acc: 0.8025
Epoch 3/10
51770/51770 [==============================] - 2s 31us/step - loss: 0.7682 - acc: 0.7465 - val_loss: 0.5782 - val_acc: 0.8278
Epoch 4/10
51770/51770 [==============================] - 2s 32us/step - loss: 0.5849 - acc: 0.8063 - val_loss: 0.4276 - val_acc: 0.8681
Epoch 5/10
51770/51770 [==============================] - 2s 31us/step - loss: 0.4712 - acc: 0.8438 - val_loss: 0.3998 - val_acc: 0.8771
Epoch 6/10
51770/51770 [==============================] - 2s 31us/step - loss: 0.3966 - acc: 0.8671 - val_loss: 0.4275 - val_acc: 0.8811
Epoch 7/10
51770/51770 [==============================] - 2s 30us/step - loss: 0.3393 - acc: 0.8860 - val_loss: 0.3272 - val_acc

In [82]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




## Sequential MLP v2, 15 Epochs, scale = 15

In [7]:
# Fix our settings

number_fruits = 100
image_scale = 15

sample_list = [.01,.02,.03,.04,.05, .1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

In [8]:
# Work through the test dataset

test_datapaths, test_labels, test_numlabels, test_images = get_image_arrays('Test', 
                                                                            number_fruits, 
                                                                            image_scale)
test_sample = []
for x in sample_list:
    test_sample.append(test_images[int(x*len(test_images))])  # takes a snapshot of a few images before flattening

for x in range(0,len(test_images)):
    test_images[x] = np.ndarray.flatten(test_images[x])

test_images = np.array(test_images)

y_test = keras.utils.to_categorical(test_numlabels, number_fruits)


Working on Test images...  Done with upload and initial processing.


In [9]:
# Work through the train dataset

train_datapaths, train_labels, train_numlabels, train_images = get_image_arrays('Training', 
                                                                                number_fruits, 
                                                                                image_scale)
train_sample = []
for x in sample_list:
    train_sample.append(train_images[int(x*len(train_images))])

for x in range(0,len(train_images)):
    train_images[x] = np.ndarray.flatten(train_images[x])

train_images = np.array(train_images)

y_train = keras.utils.to_categorical(train_numlabels, number_fruits)


Working on Training images...  Done with upload and initial processing.


In [10]:
# boil down the size of the input arrays for the model
input_shape = str(train_images[1].shape)
input_shape = input_shape.replace('(','')
input_shape = input_shape.replace(',)','')
input_shape = int(input_shape)
print(input_shape)

675


In [14]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(input_shape,)))  
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(number_fruits, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                43264     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               6500      
Total params: 53,924
Trainable params: 53,924
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model.fit(train_images, y_train,
                    batch_size=128,
                    epochs=15,
                    verbose=1,
                    validation_data=(test_images, y_test))
score = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/15
51770/51770 [==============================] - 2s 37us/step - loss: 2.6241 - acc: 0.3155 - val_loss: 1.2014 - val_acc: 0.6907
Epoch 2/15
51770/51770 [==============================] - 1s 29us/step - loss: 1.1429 - acc: 0.6389 - val_loss: 0.8778 - val_acc: 0.7520
Epoch 3/15
51770/51770 [==============================] - 2s 29us/step - loss: 0.7811 - acc: 0.7460 - val_loss: 0.5835 - val_acc: 0.8291
Epoch 4/15
51770/51770 [==============================] - 2s 32us/step - loss: 0.6071 - acc: 0.8013 - val_loss: 0.4547 - val_acc: 0.8578
Epoch 5/15
51770/51770 [==============================] - 2s 35us/step - loss: 0.4894 - acc: 0.8364 - val_loss: 0.3933 - val_acc: 0.8820
Epoch 6/15
51770/51770 [==============================] - 2s 42us/step - loss: 0.4158 - acc: 0.8604 - val_loss: 0.4007 - val_acc: 0.8733
Epoch 7/15
51770/51770 [==============================] - 2s 31us/step - loss: 0.3520 - acc: 0.8835 - val_loss: 0.4649 - val_acc

In [16]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




## Sequential MLP v3, 15 Epochs, scale = 25

In [17]:
# Fix our settings

number_fruits = 100
image_scale = 25

sample_list = [.01,.02,.03,.04,.05, .1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

In [18]:
# Work through the test dataset

test_datapaths, test_labels, test_numlabels, test_images = get_image_arrays('Test', 
                                                                            number_fruits, 
                                                                            image_scale)
test_sample = []
for x in sample_list:
    test_sample.append(test_images[int(x*len(test_images))])  # takes a snapshot of a few images before flattening

for x in range(0,len(test_images)):
    test_images[x] = np.ndarray.flatten(test_images[x])

test_images = np.array(test_images)

y_test = keras.utils.to_categorical(test_numlabels, number_fruits)


Working on Test images...  Done with upload and initial processing.


In [19]:
# Work through the train dataset

train_datapaths, train_labels, train_numlabels, train_images = get_image_arrays('Training', 
                                                                                number_fruits, 
                                                                                image_scale)
train_sample = []
for x in sample_list:
    train_sample.append(train_images[int(x*len(train_images))])

for x in range(0,len(train_images)):
    train_images[x] = np.ndarray.flatten(train_images[x])

train_images = np.array(train_images)

y_train = keras.utils.to_categorical(train_numlabels, number_fruits)


Working on Training images...  Done with upload and initial processing.


In [20]:
# boil down the size of the input arrays for the model
input_shape = str(train_images[1].shape)
input_shape = input_shape.replace('(','')
input_shape = input_shape.replace(',)','')
input_shape = int(input_shape)
print(input_shape)

1875


In [21]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(input_shape,)))  
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(number_fruits, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                120064    
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               6500      
Total params: 130,724
Trainable params: 130,724
Non-trainable params: 0
_________________________________________________________________


In [22]:
history = model.fit(train_images, y_train,
                    batch_size=128,
                    epochs=15,
                    verbose=1,
                    validation_data=(test_images, y_test))
score = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/15
51770/51770 [==============================] - 3s 62us/step - loss: 2.9514 - acc: 0.2444 - val_loss: 1.3333 - val_acc: 0.6796
Epoch 2/15
51770/51770 [==============================] - 3s 52us/step - loss: 1.4741 - acc: 0.5440 - val_loss: 0.8309 - val_acc: 0.7716
Epoch 3/15
51770/51770 [==============================] - 3s 55us/step - loss: 1.1121 - acc: 0.6479 - val_loss: 0.7837 - val_acc: 0.7604
Epoch 4/15
51770/51770 [==============================] - 3s 61us/step - loss: 0.9216 - acc: 0.7055 - val_loss: 0.6781 - val_acc: 0.8043
Epoch 5/15
51770/51770 [==============================] - 3s 53us/step - loss: 0.7964 - acc: 0.7432 - val_loss: 0.6014 - val_acc: 0.8242
Epoch 6/15
51770/51770 [==============================] - 3s 61us/step - loss: 0.7090 - acc: 0.7736 - val_loss: 0.4764 - val_acc: 0.8597
Epoch 7/15
51770/51770 [==============================] - 3s 53us/step - loss: 0.6519 - acc: 0.7914 - val_loss: 0.4863 - val_acc

In [23]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




## Seq MLP Conclusion
Decent, and very fast (once you've uploaded the images) but not in the 90%+ category yet. 

Extra Epochs seems to do little, as by #10 it seems to have reached a point of minimal or negative returns, where the random associations in each iteration cause it to "forget" how to label some fruits even as it learns how to label others.

The most critical factor seems to be the image size, as a scale of 100 pixels (the scale of the images in the download for this collection) produces abysmal results, and gets progressively better until it reaches somewhere between 10 and 20, after which further decreases drop performance (likely due to simply not having enough identifying data to go on in each image).

## Convolutional Neural Network, v1 (image_scale = 50)

In [33]:
# Fix our settings

number_fruits = 100
image_scale = 50

sample_list = [.01,.02,.03,.04,.05, .1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

In [34]:
# Work through the test dataset

test_datapaths, test_labels, test_numlabels, test_images = get_image_arrays('Test', 
                                                                            number_fruits, 
                                                                            image_scale)
test_sample = []
for x in sample_list:
    test_sample.append(test_images[int(x*len(test_images))])

for x in range(0,len(test_images)):
    test_images[x] = np.ndarray.flatten(test_images[x])

test_images = np.array(test_images)

y_test = keras.utils.to_categorical(test_numlabels, number_fruits)

test_images = test_images.reshape(test_images.shape[0], image_scale, image_scale, 3)

print('Test images processing is complete')

Working on Test images...  Done with upload and initial processing.
Test images processing is complete


In [35]:
# Work through the train dataset

train_datapaths, train_labels, train_numlabels, train_images = get_image_arrays('Training', 
                                                                                number_fruits, 
                                                                                image_scale)
train_sample = []
for x in sample_list:
    train_sample.append(train_images[int(x*len(train_images))])

for x in range(0,len(train_images)):
    train_images[x] = np.ndarray.flatten(train_images[x])

train_images = np.array(train_images)

y_train = keras.utils.to_categorical(train_numlabels, number_fruits)

train_images = train_images.reshape(train_images.shape[0], image_scale, image_scale, 3)

print('Train images processing is complete')

Working on Training images...  Done with upload and initial processing.
Train images processing is complete


In [36]:
input_shape = (image_scale, image_scale, 3)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(number_fruits, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(train_images, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(test_images, y_test))
score = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/10
51770/51770 [==============================] - 436s 8ms/step - loss: 1.8939 - acc: 0.5137 - val_loss: 0.3347 - val_acc: 0.9075
Epoch 2/10
51770/51770 [==============================] - 423s 8ms/step - loss: 0.3434 - acc: 0.8890 - val_loss: 0.1378 - val_acc: 0.9587
Epoch 3/10
51770/51770 [==============================] - 422s 8ms/step - loss: 0.1745 - acc: 0.9412 - val_loss: 0.1283 - val_acc: 0.9649
Epoch 4/10
51770/51770 [==============================] - 411s 8ms/step - loss: 0.1175 - acc: 0.9607 - val_loss: 0.0925 - val_acc: 0.9733
Epoch 5/10
51770/51770 [==============================] - 406s 8ms/step - loss: 0.0847 - acc: 0.9710 - val_loss: 0.1040 - val_acc: 0.9786
Epoch 6/10
51770/51770 [==============================] - 417s 8ms/step - loss: 0.0680 - acc: 0.9766 - val_loss: 0.0958 - val_acc: 0.9796
Epoch 7/10
51770/51770 [==============================] - 423s 8ms/step - loss: 0.0533 - acc: 0.9821 - val_loss: 0.0869 - 

## Convolutional Neural Network, v2 (image_scale = 30)

In [37]:
# Fix our settings

number_fruits = 100
image_scale = 30

sample_list = [.01,.02,.03,.04,.05, .1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

In [38]:
# Work through the test dataset

test_datapaths, test_labels, test_numlabels, test_images = get_image_arrays('Test', 
                                                                            number_fruits, 
                                                                            image_scale)
test_sample = []
for x in sample_list:
    test_sample.append(test_images[int(x*len(test_images))])

for x in range(0,len(test_images)):
    test_images[x] = np.ndarray.flatten(test_images[x])

test_images = np.array(test_images)

y_test = keras.utils.to_categorical(test_numlabels, number_fruits)

test_images = test_images.reshape(test_images.shape[0], image_scale, image_scale, 3)

print('Test images processing is complete')

Working on Test images...  Done with upload and initial processing.
Test images processing is complete


In [39]:
# Work through the train dataset

train_datapaths, train_labels, train_numlabels, train_images = get_image_arrays('Training', 
                                                                                number_fruits, 
                                                                                image_scale)
train_sample = []
for x in sample_list:
    train_sample.append(train_images[int(x*len(train_images))])

for x in range(0,len(train_images)):
    train_images[x] = np.ndarray.flatten(train_images[x])

train_images = np.array(train_images)

y_train = keras.utils.to_categorical(train_numlabels, number_fruits)

train_images = train_images.reshape(train_images.shape[0], image_scale, image_scale, 3)

print('Train images processing is complete')

Working on Training images...  Done with upload and initial processing.
Train images processing is complete


In [40]:
input_shape = (image_scale, image_scale, 3)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(number_fruits, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(train_images, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(test_images, y_test))
score = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/10
51770/51770 [==============================] - 139s 3ms/step - loss: 1.7889 - acc: 0.5337 - val_loss: 0.2749 - val_acc: 0.9141
Epoch 2/10
51770/51770 [==============================] - 138s 3ms/step - loss: 0.2570 - acc: 0.9177 - val_loss: 0.1699 - val_acc: 0.9461
Epoch 3/10
51770/51770 [==============================] - 137s 3ms/step - loss: 0.1233 - acc: 0.9595 - val_loss: 0.1078 - val_acc: 0.9639
Epoch 4/10
51770/51770 [==============================] - 136s 3ms/step - loss: 0.0813 - acc: 0.9729 - val_loss: 0.1152 - val_acc: 0.9611
Epoch 5/10
51770/51770 [==============================] - 136s 3ms/step - loss: 0.0611 - acc: 0.9798 - val_loss: 0.0749 - val_acc: 0.9776
Epoch 6/10
51770/51770 [==============================] - 137s 3ms/step - loss: 0.0481 - acc: 0.9836 - val_loss: 0.0714 - val_acc: 0.9784
Epoch 7/10
51770/51770 [==============================] - 137s 3ms/step - loss: 0.0392 - acc: 0.9868 - val_loss: 0.0988 - 

## Convolutional Neural Network, v3 (image_scale = 30, kernels = 4x4)

In [83]:
# Fix our settings

number_fruits = 100
image_scale = 30

sample_list = [.01,.02,.03,.04,.05, .1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

In [84]:
# Work through the test dataset

test_datapaths, test_labels, test_numlabels, test_images = get_image_arrays('Test', 
                                                                            number_fruits, 
                                                                            image_scale)
test_sample = []
for x in sample_list:
    test_sample.append(test_images[int(x*len(test_images))])

for x in range(0,len(test_images)):
    test_images[x] = np.ndarray.flatten(test_images[x])

test_images = np.array(test_images)

y_test = keras.utils.to_categorical(test_numlabels, number_fruits)

test_images = test_images.reshape(test_images.shape[0], image_scale, image_scale, 3)

print('Test images processing is complete')

Working on Test images...  Done with upload and initial processing.
Test images processing is complete


In [85]:
# Work through the train dataset

train_datapaths, train_labels, train_numlabels, train_images = get_image_arrays('Training', 
                                                                                number_fruits, 
                                                                                image_scale)
train_sample = []
for x in sample_list:
    train_sample.append(train_images[int(x*len(train_images))])

for x in range(0,len(train_images)):
    train_images[x] = np.ndarray.flatten(train_images[x])

train_images = np.array(train_images)

y_train = keras.utils.to_categorical(train_numlabels, number_fruits)

train_images = train_images.reshape(train_images.shape[0], image_scale, image_scale, 3)

print('Train images processing is complete')

Working on Training images...  Done with upload and initial processing.
Train images processing is complete


In [86]:
input_shape = (image_scale, image_scale, 3)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(4, 4),
                 activation='relu',
                 input_shape=input_shape,))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(number_fruits, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(train_images, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(test_images, y_test))
score = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/10
51770/51770 [==============================] - 193s 4ms/step - loss: 1.7834 - acc: 0.5497 - val_loss: 0.2610 - val_acc: 0.9243
Epoch 2/10
51770/51770 [==============================] - 193s 4ms/step - loss: 0.2243 - acc: 0.9289 - val_loss: 0.1503 - val_acc: 0.9517
Epoch 3/10
51770/51770 [==============================] - 200s 4ms/step - loss: 0.1078 - acc: 0.9639 - val_loss: 0.1262 - val_acc: 0.9639
Epoch 4/10
51770/51770 [==============================] - 188s 4ms/step - loss: 0.0732 - acc: 0.9761 - val_loss: 0.1187 - val_acc: 0.9655
Epoch 5/10
51770/51770 [==============================] - 191s 4ms/step - loss: 0.0519 - acc: 0.9828 - val_loss: 0.0953 - val_acc: 0.9746
Epoch 6/10
51770/51770 [==============================] - 191s 4ms/step - loss: 0.0411 - acc: 0.9863 - val_loss: 0.0978 - val_acc: 0.9732
Epoch 7/10
51770/51770 [==============================] - 187s 4ms/step - loss: 0.0343 - acc: 0.9887 - val_loss: 0.0960 - 

In [87]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




## Convolutional Neural Network, v4 (image_scale = 30, kernels = 3x3, epochs = 15)

In [24]:
# Fix our settings

number_fruits = 100
image_scale = 30

sample_list = [.01,.02,.03,.04,.05, .1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

In [25]:
# Work through the test dataset

test_datapaths, test_labels, test_numlabels, test_images = get_image_arrays('Test', 
                                                                            number_fruits, 
                                                                            image_scale)
test_sample = []
for x in sample_list:
    test_sample.append(test_images[int(x*len(test_images))])

for x in range(0,len(test_images)):
    test_images[x] = np.ndarray.flatten(test_images[x])

test_images = np.array(test_images)

y_test = keras.utils.to_categorical(test_numlabels, number_fruits)

test_images = test_images.reshape(test_images.shape[0], image_scale, image_scale, 3)

print('Test images processing is complete')

Working on Test images...  Done with upload and initial processing.
Test images processing is complete


In [26]:
# Work through the train dataset

train_datapaths, train_labels, train_numlabels, train_images = get_image_arrays('Training', 
                                                                                number_fruits, 
                                                                                image_scale)
train_sample = []
for x in sample_list:
    train_sample.append(train_images[int(x*len(train_images))])

for x in range(0,len(train_images)):
    train_images[x] = np.ndarray.flatten(train_images[x])

train_images = np.array(train_images)

y_train = keras.utils.to_categorical(train_numlabels, number_fruits)

train_images = train_images.reshape(train_images.shape[0], image_scale, image_scale, 3)

print('Train images processing is complete')

Working on Training images...  Done with upload and initial processing.
Train images processing is complete


In [27]:
input_shape = (image_scale, image_scale, 3)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(number_fruits, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(train_images, y_train,
          batch_size=128,
          epochs=15,
          verbose=1,
          validation_data=(test_images, y_test))
score = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/15
51770/51770 [==============================] - 155s 3ms/step - loss: 1.8415 - acc: 0.5223 - val_loss: 0.3420 - val_acc: 0.8959
Epoch 2/15
51770/51770 [==============================] - 186s 4ms/step - loss: 0.2792 - acc: 0.9084 - val_loss: 0.1269 - val_acc: 0.9625
Epoch 3/15
51770/51770 [==============================] - 191s 4ms/step - loss: 0.1368 - acc: 0.9545 - val_loss: 0.0990 - val_acc: 0.9694
Epoch 4/15
51770/51770 [==============================] - 196s 4ms/step - loss: 0.0895 - acc: 0.9706 - val_loss: 0.0735 - val_acc: 0.9791
Epoch 5/15
51770/51770 [==============================] - 163s 3ms/step - loss: 0.0677 - acc: 0.9776 - val_loss: 0.0832 - val_acc: 0.9765
Epoch 6/15
51770/51770 [==============================] - 156s 3ms/step - loss: 0.0523 - acc: 0.9826 - val_loss: 0.0727 - val_acc: 0.9808
Epoch 7/15
51770/51770 [==============================] - 163s 3ms/step - loss: 0.0441 - acc: 0.9854 - val_loss: 0.0560 - 

In [28]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




## Convolutional Neural Network, v5 (image_scale = 20)

In [29]:
# Fix our settings

number_fruits = 100
image_scale = 20

sample_list = [.01,.02,.03,.04,.05, .1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

In [30]:
# Work through the test dataset

test_datapaths, test_labels, test_numlabels, test_images = get_image_arrays('Test', 
                                                                            number_fruits, 
                                                                            image_scale)
test_sample = []
for x in sample_list:
    test_sample.append(test_images[int(x*len(test_images))])

for x in range(0,len(test_images)):
    test_images[x] = np.ndarray.flatten(test_images[x])

test_images = np.array(test_images)

y_test = keras.utils.to_categorical(test_numlabels, number_fruits)

test_images = test_images.reshape(test_images.shape[0], image_scale, image_scale, 3)

print('Test images processing is complete')

Working on Test images...  Done with upload and initial processing.
Test images processing is complete


In [31]:
# Work through the train dataset

train_datapaths, train_labels, train_numlabels, train_images = get_image_arrays('Training', 
                                                                                number_fruits, 
                                                                                image_scale)
train_sample = []
for x in sample_list:
    train_sample.append(train_images[int(x*len(train_images))])

for x in range(0,len(train_images)):
    train_images[x] = np.ndarray.flatten(train_images[x])

train_images = np.array(train_images)

y_train = keras.utils.to_categorical(train_numlabels, number_fruits)

train_images = train_images.reshape(train_images.shape[0], image_scale, image_scale, 3)

print('Train images processing is complete')

Working on Training images...  Done with upload and initial processing.
Train images processing is complete


In [32]:
input_shape = (image_scale, image_scale, 3)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(number_fruits, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(train_images, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(test_images, y_test))
score = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/10
51770/51770 [==============================] - 71s 1ms/step - loss: 1.8746 - acc: 0.5093 - val_loss: 0.3800 - val_acc: 0.8915
Epoch 2/10
51770/51770 [==============================] - 68s 1ms/step - loss: 0.3172 - acc: 0.8980 - val_loss: 0.1663 - val_acc: 0.9432
Epoch 3/10
51770/51770 [==============================] - 74s 1ms/step - loss: 0.1568 - acc: 0.9491 - val_loss: 0.1471 - val_acc: 0.9560
Epoch 4/10
51770/51770 [==============================] - 64s 1ms/step - loss: 0.1022 - acc: 0.9659 - val_loss: 0.0962 - val_acc: 0.9743
Epoch 5/10
51770/51770 [==============================] - 63s 1ms/step - loss: 0.0780 - acc: 0.9738 - val_loss: 0.0935 - val_acc: 0.9723
Epoch 6/10
51770/51770 [==============================] - 64s 1ms/step - loss: 0.0612 - acc: 0.9794 - val_loss: 0.0797 - val_acc: 0.9770
Epoch 7/10
51770/51770 [==============================] - 63s 1ms/step - loss: 0.0504 - acc: 0.9831 - val_loss: 0.0889 - val_acc

In [33]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




## CNN conclusions:
The most effective of the three methods, though also the most time-intensive.

Increasing the kernel size seemed to reduce performance.  Optimal image size was somewhere in the vicinity of 30x30.  An increase in the number of epochs either reduced accuracy or had no effect other than to allow randomness to drop the result by a few fractions of a percent.

## Recurrent Neural Network, v1 (image_scale = 30, Epochs = 3)

In [34]:
# Fix our settings

number_fruits = 100
image_scale = 30

sample_list = [.01,.02,.03,.04,.05, .1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

In [35]:
# Work through the test dataset

test_datapaths, test_labels, test_numlabels, test_images = get_image_arrays('Test', 
                                                                            number_fruits, 
                                                                            image_scale)
test_sample = []
for x in sample_list:
    test_sample.append(test_images[int(x*len(test_images))])

for x in range(0,len(test_images)):
    test_images[x] = np.ndarray.flatten(test_images[x])

test_images = np.array(test_images)

y_test = keras.utils.to_categorical(test_numlabels, number_fruits)

test_images = test_images.reshape(test_images.shape[0], image_scale, image_scale, 3)

print('Test images processing is complete')

Working on Test images...  Done with upload and initial processing.
Test images processing is complete


In [36]:
# Work through the train dataset

train_datapaths, train_labels, train_numlabels, train_images = get_image_arrays('Training', 
                                                                                number_fruits, 
                                                                                image_scale)
train_sample = []
for x in sample_list:
    train_sample.append(train_images[int(x*len(train_images))])

for x in range(0,len(train_images)):
    train_images[x] = np.ndarray.flatten(train_images[x])

train_images = np.array(train_images)

y_train = keras.utils.to_categorical(train_numlabels, number_fruits)

train_images = train_images.reshape(train_images.shape[0], image_scale, image_scale, 3)

print('Train images processing is complete')

Working on Training images...  Done with upload and initial processing.
Train images processing is complete


In [37]:

# Training parameters.
batch_size = 64
# num_classes = number_fruits
epochs = 3

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# Reshapes data to 4D for Hierarchical RNN.
print('x_train shape:', train_images.shape)
print(train_images.shape[0], 'train samples')
print(test_images.shape[0], 'test samples')

row, col, pixel = train_images.shape[1:]


x_train shape: (51770, 30, 30, 3)
51770 train samples
17376 test samples


In [38]:

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(number_fruits, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(train_images, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_images, y_test))

# Evaluation.
scores = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/3
51770/51770 [==============================] - 152s 3ms/step - loss: 3.5909 - acc: 0.1292 - val_loss: 2.7050 - val_acc: 0.3187
Epoch 2/3
51770/51770 [==============================] - 148s 3ms/step - loss: 2.1660 - acc: 0.4402 - val_loss: 1.8487 - val_acc: 0.4695
Epoch 3/3
51770/51770 [==============================] - 146s 3ms/step - loss: 1.3842 - acc: 0.6448 - val_loss: 1.2616 - val_acc: 0.6901
Test loss: 1.261625601853455
Test accuracy: 0.6900897790055248


In [39]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




## Recurrent Neural Network v2, Epochs = 15, Scale = 30

In [40]:

# Training parameters.
batch_size = 64
# num_classes = number_fruits
epochs = 15

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# Reshapes data to 4D for Hierarchical RNN.
print('x_train shape:', train_images.shape)
print(train_images.shape[0], 'train samples')
print(test_images.shape[0], 'test samples')

row, col, pixel = train_images.shape[1:]


x_train shape: (51770, 30, 30, 3)
51770 train samples
17376 test samples


In [41]:

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(number_fruits, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(train_images, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_images, y_test))

# Evaluation.
scores = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/15
51770/51770 [==============================] - 154s 3ms/step - loss: 3.5835 - acc: 0.1400 - val_loss: 2.7246 - val_acc: 0.3231
Epoch 2/15
51770/51770 [==============================] - 150s 3ms/step - loss: 2.1960 - acc: 0.4303 - val_loss: 1.8402 - val_acc: 0.5147
Epoch 3/15
51770/51770 [==============================] - 154s 3ms/step - loss: 1.4594 - acc: 0.6189 - val_loss: 1.4940 - val_acc: 0.5858
Epoch 4/15
51770/51770 [==============================] - 150s 3ms/step - loss: 0.9828 - acc: 0.7488 - val_loss: 0.9887 - val_acc: 0.7278
Epoch 5/15
51770/51770 [==============================] - 167s 3ms/step - loss: 0.6737 - acc: 0.8279 - val_loss: 0.7790 - val_acc: 0.7967
Epoch 6/15
51770/51770 [==============================] - 157s 3ms/step - loss: 0.4690 - acc: 0.8786 - val_loss: 0.6289 - val_acc: 0.8231
Epoch 7/15
51770/51770 [==============================] - 160s 3ms/step - loss: 0.3462 - acc: 0.9087 - val_loss: 0.4708 - 

In [42]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




## Recurrent Neural Network v3, Epochs = 10, scale = 20

In [43]:
# Fix our settings

number_fruits = 100
image_scale = 20

sample_list = [.01,.02,.03,.04,.05, .1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

In [44]:
# Work through the test dataset

test_datapaths, test_labels, test_numlabels, test_images = get_image_arrays('Test', 
                                                                            number_fruits, 
                                                                            image_scale)
test_sample = []
for x in sample_list:
    test_sample.append(test_images[int(x*len(test_images))])

for x in range(0,len(test_images)):
    test_images[x] = np.ndarray.flatten(test_images[x])

test_images = np.array(test_images)

y_test = keras.utils.to_categorical(test_numlabels, number_fruits)

test_images = test_images.reshape(test_images.shape[0], image_scale, image_scale, 3)

print('Test images processing is complete')

Working on Test images...  Done with upload and initial processing.
Test images processing is complete


In [45]:
# Work through the train dataset

train_datapaths, train_labels, train_numlabels, train_images = get_image_arrays('Training', 
                                                                                number_fruits, 
                                                                                image_scale)
train_sample = []
for x in sample_list:
    train_sample.append(train_images[int(x*len(train_images))])

for x in range(0,len(train_images)):
    train_images[x] = np.ndarray.flatten(train_images[x])

train_images = np.array(train_images)

y_train = keras.utils.to_categorical(train_numlabels, number_fruits)

train_images = train_images.reshape(train_images.shape[0], image_scale, image_scale, 3)

print('Train images processing is complete')

Working on Training images...  Done with upload and initial processing.
Train images processing is complete


In [46]:

# Training parameters.
batch_size = 64
# num_classes = number_fruits
epochs = 10

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# Reshapes data to 4D for Hierarchical RNN.
print('x_train shape:', train_images.shape)
print(train_images.shape[0], 'train samples')
print(test_images.shape[0], 'test samples')

row, col, pixel = train_images.shape[1:]


x_train shape: (51770, 20, 20, 3)
51770 train samples
17376 test samples


In [47]:

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(number_fruits, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(train_images, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_images, y_test))

# Evaluation.
scores = model.evaluate(test_images, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 51770 samples, validate on 17376 samples
Epoch 1/10
51770/51770 [==============================] - 83s 2ms/step - loss: 3.4980 - acc: 0.1656 - val_loss: 2.3979 - val_acc: 0.4134
Epoch 2/10
51770/51770 [==============================] - 93s 2ms/step - loss: 1.8562 - acc: 0.5359 - val_loss: 1.6999 - val_acc: 0.5165
Epoch 3/10
51770/51770 [==============================] - 81s 2ms/step - loss: 1.1135 - acc: 0.7228 - val_loss: 1.0869 - val_acc: 0.7203
Epoch 4/10
51770/51770 [==============================] - 74s 1ms/step - loss: 0.7040 - acc: 0.8239 - val_loss: 0.7144 - val_acc: 0.8147
Epoch 5/10
51770/51770 [==============================] - 76s 1ms/step - loss: 0.4747 - acc: 0.8832 - val_loss: 1.0376 - val_acc: 0.6896
Epoch 6/10
51770/51770 [==============================] - 71s 1ms/step - loss: 0.3354 - acc: 0.9169 - val_loss: 0.5667 - val_acc: 0.8371
Epoch 7/10
51770/51770 [==============================] - 71s 1ms/step - loss: 0.2451 - acc: 0.9406 - val_loss: 0.4391 - val_acc

In [48]:
os.system('say "all done."'); print('\a')  # this could take a while, let me know when it's done




## RNN conclusions:
Almost as time-intensive as CNN, but with less accuracy to show for it.

Though increasing the number of epochs did improve performance, accuracy gains leveled off around the 90th percentile, and then proceeded to fall.

Dropping the image scale to 20x20 improved performance, as expected, though I'm uncertain how much of the sudden spike in val_acc in epoch 10 of v3 was due to a lucky random fluke or actual improvement in performance.  Epoch 9 saw a *drop* in val_acc, to .88, which then bounced up to .94 for epoch 10.  On the other hand, the accuracy (acc, not val_acc) of the model had steadily increased throughout.  The combination of these two implies to me that the model *was* becoming more accurate, and the last epoch simply saw a marked reduction in overfitting.